In [1]:
from langchain.chat_models import init_chat_model
from langchain_postgres import PGVector
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.tools import tool
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
if not os.environ.get("GOOGLE_API_KEY"):
  print("Please set the GOOGLE_API_KEY environment variable.")

model = init_chat_model("google_genai:gemini-2.5-flash")

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [3]:
if not os.environ.get("SUPABASE_CONNECTION_STRING"):
    print("Please set the SUPABASE_CONNECTION_STRING environment variable.")

vector_store = PGVector(
    embeddings=embeddings,
    collection_name="lilianweng_blog",
    connection=os.environ["SUPABASE_CONNECTION_STRING"],
)

In [4]:
@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=4)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [5]:
from langchain.agents import create_agent


tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)

In [8]:
query = (
    "Wat zijn de uitdagingen en limieten bij het bouwen van een LLM agent?"
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    last_message = event["messages"][-1]
    last_message.pretty_print()

================================ Human Message =================================

Wat zijn de uitdagingen en limieten bij het bouwen van een LLM agent?
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (c51ee18b-6404-4c13-8f87-37b60dbe89cc)
 Call ID: c51ee18b-6404-4c13-8f87-37b60dbe89cc
  Args:
    query: uitdagingen en limieten LLM agent
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 39137}
Content: Finite context length: The restricted context capacity limits the inclusion of historical information, detailed instructions, API call context, and responses. The design of the system has to work with this limited communication bandwidth, while mechanisms like self-reflection to learn from past mistakes would benefit a lot from long or infinite context windows. Although vector stores

In [9]:
# Render the agent's last message as Markdown in the Jupyter output
from IPython.display import Markdown, display
display(Markdown(last_message.text))

Bij het bouwen van een LLM-agent zijn er verschillende uitdagingen en beperkingen:

1.  **Beperkte contextlengte:** De eindige contextlengte beperkt de hoeveelheid historische informatie, gedetailleerde instructies, API-aanroepcontext en antwoorden die kunnen worden opgenomen. Dit bemoeilijkt het vermogen van de agent om te leren van fouten uit het verleden door middel van zelfreflectie.
2.  **Uitdagingen bij lange-termijnplanning en taakdecompositie:** LLM's hebben moeite met plannen over een lange geschiedenis en het effectief verkennen van de oplossingsruimte. Ze vinden het ook moeilijk om plannen aan te passen wanneer er onverwachte fouten optreden, waardoor ze minder robuust zijn dan mensen.
3.  **Betrouwbaarheid van de natuurlijke taalinterface:** LLM's kunnen opmaakfouten maken of "rebels gedrag" vertonen (bijvoorbeeld weigeren instructies op te volgen), wat de betrouwbaarheid van hun uitvoer beïnvloedt bij interactie met externe componenten zoals geheugen en tools. Dit vereist vaak veel moeite bij het parsen van de modeluitvoer.